<a href="https://colab.research.google.com/github/IKeeso/Project9_data/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stanza
!pip install conllu
!git clone https://github.com/IKeeso/Project9_data.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 11.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [2]:
import stanza
from stanza.utils.conll import CoNLL
from conllu import parse
from conllu import TokenList
import re

!ls Project9_data


expressen.txt  Project.ipynb


In [3]:
stanza.download('sv')
nlp = stanza.Pipeline(lang='sv')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: sv (Swedish) ...


INFO:stanza:Downloaded file to /root/stanza_resources/sv/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: sv (Swedish):
| Processor | Package            |
----------------------------------
| tokenize  | talbanken          |
| pos       | talbanken_charlm   |
| lemma     | talbanken_nocharlm |
| depparse  | talbanken_charlm   |
| ner       | suc3shuffle        |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [4]:
def tag_and_convert(filename):
  %cd /content/Project9_data
  try:
    with open(filename, 'r', encoding='utf-8') as file:
      text = file.read()
      file.close()
  except:
    text = filename
  doc = nlp(text)
  pathname = filename.split(".")[0]
  conllufilename = f"{pathname}.conllu"
  CoNLL.write_doc2conll(doc, conllufilename)

In [5]:
tag_and_convert("expressen.txt")
tag_and_convert("Personer som är på spektrumet. Personer på spektrumet. Personer som har autism")

/content/Project9_data
/content/Project9_data


In [6]:
"""
!git remote -v
!git init
!git config --global user.email 'isali352@student.liu.se'
!git config --global user.name 'IKeeso'
!git add expressen.conllu
!git commit -m 'New commit'  # commit in Colab
!git push origin main
"""

"\n!git remote -v\n!git init\n!git config --global user.email 'isali352@student.liu.se'\n!git config --global user.name 'IKeeso'\n!git add expressen.conllu\n!git commit -m 'New commit'  # commit in Colab\n!git push origin main\n"

In [46]:
class SemanticAnalyzer:
    def __init__(self, file_path):
        self.file_path = file_path
        self.data = self.parse_conll_u(self.file_path)
        self.autism_search_string = re.compile(r"^.*\baut(ism|istisk)\s[A-Ö]*\s", re.IGNORECASE)
        #self.person_first_search_string = re.compile(r"^.*\(PRON|NOUN|PROPN)\s[A-Ö]*\s\(PP|VB|ADP)\s\(NOUN|nmod)", re.IGNORECASE)
        self.sentence_matches = []

    def parse_conll_u(self, file_path):
      with open(file_path, 'r', encoding='utf-8') as file:
        data = {}
        sentence = []
        for line in file:
          line = line.strip()
          if line.startswith('# text'):
            sentence = []
          if line.startswith("# sent_id"):
            line = line.split("=")
            key = line[1].strip()
            sentence = []
          elif line == '':
            if sentence:
              data[key] = sentence
          else:
              sentence.append(line.split('\t'))
      return data

    def person_first_lang(self, data):
      PFL_sents = {}
      PFL_regx = re.compile(r"(nsubj\s|NOUN\s).*(case|acl:relcl\s).*(nmod|obj|obl)", re.IGNORECASE)
      for id, sentence in data.items():
        xpos = ' '.join(token[7] if token[7] != "root" else token[3] for token in sentence)
        if PFL_regx.match(xpos):
          PFL_sents[id] = xpos
      return PFL_sents

    def regx_analyze_sentence(self, sentence, search_string):
      words = ' '.join(token[1] for token in sentence)
      if search_string.match(words):
        self.sentence_matches.append(sentence)
        return True
      else:
        return False

# Example usage:
analyzer = SemanticAnalyzer(file_path='expressen.conllu')
tester = SemanticAnalyzer(file_path='Personer som är på spektrumet.conllu')

for id in analyzer.person_first_lang(analyzer.data).keys():
  print(" ".join(token[1] for token in analyzer.data[id]))
  print("")


#for id, sentence in analyzer.data.items():
#  if analyzer.regx_analyze_sentence(analyzer.data[id], analyzer.autism_search_string):
#    print(analyzer.person_first_lang(analyzer.data[id]))
#    print(" ".join(token[1]for token in sentence))
#    print("")



#for id, sentence in analyzer.data.items():
#  if analyzer.regx_analyze_sentence(sentence, analyzer.autism_search_string):
#    print(f"(sent_id: {id})", ' '.join(token[1] for token in sentence), "\n")

Många med autism känner sig missförstådda och har svårt att klara skolan .

Vi behöver en skola som ger alla elever möjlighet att lyckas utifrån sina egna förutsättningar , säger Linn Rivero Mildén som är kommunikatör på Autism Sverige .

Vi kan dessutom sprida kunskap genom att producera Podden Autism och vi kan arbeta för en skola för alla , berättar Linn .

Mildén som är kommunikatör på Autism Sverige .

Didriksson från Målilla är lottköpare och även vinnare i Postkodlotteriet .

Hon fick möjlighet att följa med Postkodlotteriets vinstutdelare Sandra Dahlberg när hon nyligen besökte Autism Sverige för att dela ut årets basstödscheck .

Didriksson från Målilla är lottköpare och även vinnare i Postkodlotteriet .

Hon berättar om sin barndom i den lilla byn Klimpfjäll där hon gick i skolan i en klass med bara fem barn .

Linn trycker på vikten av att utbilda skolpersonal och öka kunskapen om autism i skolorna så att alla så tidigt som möjligt kan mötas med förståelse och få rätt föruts